In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from lore_sa.dataset import TabularDataset
import pandas as pd

In [ ]:
dataset = TabularDataset.from_csv('test/resources/adult.csv', class_name = "class")
dataset.df.dropna(inplace = True)
dataset.df

In [ ]:
dataset.descriptor

In [ ]:
dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.update_descriptor()

In [ ]:
dataset.descriptor['categoric'].keys()

In [ ]:
from lore_sa.encoder_decoder import TabularEnc
encoder = TabularEnc(dataset.descriptor)

encoded = []
for x in dataset.df.iloc:
    
    
    encoded.append(encoder.encode(x.values))
    
    


In [ ]:
encoder.decode(encoded[0])

In [ ]:
encoder.dataset_descriptor

In [ ]:
encoder.get_encoded_features()

In [ ]:
encoded_df =  pd.DataFrame(encoded, columns = [encoder.encoded_features[i] for i in range(len(encoded[0]))])
encoded_df

In [ ]:
# random generation

features = [c for c in encoded_df.columns if c != dataset.class_name]

x = encoded_df[features].iloc[10].values

from lore_sa.neighgen.random import RandomGenerator

gen = RandomGenerator()

neighbour = gen.generate(x,10000, dataset.descriptor, onehotencoder = encoder)

neighbour.df

## creating a blackbox


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

feature_names = [c for c in encoded_df.columns if c != 'class']
class_name = dataset.class_name
test_size= 0.3
random_state= 42

X_train, X_test, Y_train, Y_test = train_test_split(encoded_df[feature_names].values, encoded_df[class_name].values,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    stratify=encoded_df[class_name].values)




# Train a random forest model

bb = RandomForestClassifier(n_estimators=20, random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)



# example of predictions
Y_pred = bb.predict(X_test)

print('Accuracy %.3f' % accuracy_score(Y_test, Y_pred))
print('F1-measure %.3f' % f1_score(Y_test, Y_pred))






### neighbour classification

In [ ]:
from lore_sa.bbox import sklearn_classifier_bbox

bbox = sklearn_classifier_bbox.sklearnBBox(bb)

neighbour.df['class'] = bbox.predict(neighbour.df[features])
neighbour.set_class_name('class')

## surrogate

In [ ]:
from lore_sa.surrogate import DecisionTreeSurrogate

surrogate = DecisionTreeSurrogate()


surrogate.train(neighbour.df[features].values , neighbour.df['class'] )

In [ ]:
x = encoded_df[features].iloc[10].values


rule = surrogate.get_rule(x, neighbour, encoder)

print (rule)

In [ ]:
encoder

In [ ]:
import numpy as np
features
neighbour.df[[c for c in features if c!='class'][0]]

In [ ]:
crules, deltas = surrogate.get_counterfactual_rules(x=x, y = 'class', Y=features, neighborhood_dataset = neighbour, encdec = encoder)